# GeoSpatial Data Project

In [1]:
import src.funciones_llamada_api as fla
import os
from dotenv import load_dotenv

In [2]:
# Librerias
import pandas as pd

In [3]:
# Librerias visualizacion json
import pyjsonviewer

### asignamos los token de la carpeta .env a unas varibles que creamos

In [4]:
load_dotenv()

# asignamos los token de la carpeta .env a unas varibles que creamos
token = os.getenv("Client_Id")
token2 = os.getenv("Client_Secret")

#### con geocode buscamos la longitud y la latitud de la direccion que le demos

In [5]:
dir_madrid = "Plaza de Colón, Madrid"
dir_alcobendas = "Heron Diversia, Avenida de Bruselas, Alcobendas"
dir_rozas = "Heron City Las Rozas, Calle Camilo José Cela, Las Rozas de Madrid"

In [6]:
# Llamamos a la funcion que nos pasa las coordenadas con geocode
madrid = fla.geocode_coord(dir_madrid)
alcobendas = fla.geocode_coord(dir_alcobendas)
rozas = fla.geocode_coord(dir_rozas)

##### funcion generar_json y obtener_datos

In [7]:
resp = fla.generar_json(madrid, "vegan", token, token2)

In [8]:
datos_finales = fla.obtener_datos(resp)

In [11]:
type(datos_finales)

dict

In [13]:
#from functools import reduce
#import operator

In [24]:
mapa_nombre = ["venue", "name"]
mapa_latitud = ["venue", "location", "lat"]
mapa_longitud = ["venue", "location", "lng"]

In [12]:
#def getFromDict(diccionario, mapa):
#    return reduce(operator.getitem,mapa,diccionario)

In [26]:
def type_point(lista):
    return {"type": "Point", 
            "coordinates": lista }

In [27]:
unjsonnuevo = []
for dic in datos_finales:
    paralista= {}
    paralista["nombre"]= getFromDict(dic,mapa_nombre)
    paralista["latitud"]= getFromDict(dic,mapa_latitud)
    paralista["longitud"]= getFromDict(dic,mapa_longitud)
    paralista["location"]=  type_point([paralista["longitud"],paralista["latitud"]])
    unjsonnuevo.append(paralista)

In [31]:
unjsonnuevo[0]

{'nombre': 'La Terraza de Las Rozas Village',
 'latitud': 40.518263334929124,
 'longitud': -3.9031637657058136,
 'location': {'type': 'Point',
  'coordinates': [-3.9031637657058136, 40.518263334929124]}}

#### exportamos el diccionaro a un json

In [32]:
# Exportamos el json con json.dump y luego .view_data(json_file="ruta")
import json
with open('prueba.json', 'w') as f:
    json.dump(unjsonnuevo, f)